# Re-ranking semántico

La **búsqueda por palabras clave**, comúnmente utilizada en motores de búsqueda, prioriza la similitud de palabras entre la consulta del usuario y los documentos disponibles, pero suele ignorar la **semántica** de las palabras. Por ejemplo, no distinguiría entre "banco" como institución financiera y "banco" como asiento, ya que no comprende las diferencias contextuales y semánticas entre palabras. Por el contrario, la búsqueda semántica sí entiende estas diferencias, brindando resultados más precisos y relevantes.

Migrar completamente a sistemas de búsqueda semántica puede ser un reto para muchas empresas debido a la profunda integración de los sistemas basados en palabras clave. Una solución es **re-rankear** los resultados de la búsqueda por palabras clave usando un modelo de búsqueda semántica basado en **word embeddings**.

**Cohere Rerank** ofrece una solución de re-ranking semántico fácil de integrar en sistemas existentes, necesitando solo unas pocas líneas de código, permitiendo así una transición suave hacia métodos de búsqueda más avanzados y precisos.

Para obtener más información sobre `Cohere Rerank` y cómo puede beneficiar a tu sistema de búsqueda, te invito a visitar su [blog](https://txt.cohere.com/rerank/).


> Al integrar la precisión contextual de la búsqueda semántica con la eficacia de la búsqueda por palabras clave, podemos superar las limitaciones inherentes de cada método, logrando así resultados de búsqueda más relevantes, precisos y ricos en contexto.

## Librerías

In [1]:
import cohere
from dotenv import load_dotenv
from langchain.retrievers import BM25Retriever
from langchain.schema import Document

from src.langchain_docs_loader import load_langchain_docs_splitted

load_dotenv()

True

## Carga de datos

In [2]:
docs = load_langchain_docs_splitted()

## Creación de herramienta de búsqueda por palabras clave

BM25 es una función de ranking avanzada usada para clasificar documentos en sistemas de recuperación de información, basándose en su relevancia respecto a una consulta de búsqueda. A diferencia de la búsqueda por palabras clave básica, que solo considera la presencia o ausencia de palabras, BM25 calcula un score de relevancia para cada documento, teniendo en cuenta la frecuencia de aparición del término y su rareza en la colección de documentos.

In [3]:
keywordk_retriever = BM25Retriever.from_documents(docs)


def keyword_document_search(query: str, k: int) -> list[Document]:
    keywordk_retriever.k = k
    return keywordk_retriever.get_relevant_documents(query)

## Búsqueda de documentos relevantes por palabras clave

In [4]:
relevant_keyword_documents = keyword_document_search(
    query="How to integrate LCEL into my Retrieval augmented generation system with a keyword search retriever?",
    k=100,
)

print("Keyword search results:")
for i, document in enumerate(relevant_keyword_documents):
    print(f"{i+1}. {document.metadata['source']}")

Keyword search results:
1. https://python.langchain.com/docs/integrations/memory/remembrall
2. https://python.langchain.com/docs/expression_language/
3. https://python.langchain.com/docs/modules/memory/adding_memory
4. https://python.langchain.com/docs/expression_language/cookbook/
5. https://python.langchain.com/docs/modules/memory/
6. https://python.langchain.com/docs/additional_resources/tutorials
7. https://python.langchain.com/docs/additional_resources/tutorials
8. https://python.langchain.com/docs/use_cases/question_answering/how_to/code/twitter-the-algorithm-analysis-deeplake
9. https://python.langchain.com/docs/integrations/memory/motorhead_memory
10. https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db
11. https://python.langchain.com/docs/guides/deployments/template_repos
12. https://python.langchain.com/docs/integrations/vectorstores/elasticsearch
13. https://python.langchain.com/docs/use_cases/question_answering/how_to/flare
14. https://pytho

## Re-ranking semántico de los documentos relevantes

Una vez que hemos obtenido los documentos más relevantes para nuestra consulta de búsqueda, podemos re-rankearlos usando un modelo de búsqueda semántica basado en **word embeddings**.

En este caso, utilizaremos `Cohere Rerank` para obtener los documentos más relevantes para nuestra consulta de búsqueda, re-rankeando los documentos obtenidos por BM25.

Para que `Cohere Rerank` funcione, necesitarás una cuenta de `Cohere` y un `API key`. Puedes obtener tu `API key` [aquí](https://dashboard.cohere.com/api-keys).

In [5]:
co = cohere.Client()

In [6]:
reranked_hits = co.rerank(
    query="How to integrate LCEL into my Retrieval augmented generation system with a keyword search retriever?",
    documents=[doc.page_content for doc in relevant_keyword_documents],
    top_n=10,
    model="rerank-multilingual-v2.0",
)

print("Reranked results:")
for hit in reranked_hits:
    print(relevant_keyword_documents[hit.index].metadata["source"])

Reranked results:
https://python.langchain.com/docs/expression_language/cookbook/
https://python.langchain.com/docs/use_cases/question_answering/how_to/flare
https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db
https://python.langchain.com/docs/modules/data_connection/
https://python.langchain.com/docs/modules/agents/agent_types/chat_conversation_agent
https://python.langchain.com/docs/integrations/vectorstores/neo4jvector
https://python.langchain.com/docs/use_cases/question_answering/how_to/multi_retrieval_qa_router
https://python.langchain.com/docs/integrations/memory/remembrall
https://python.langchain.com/docs/integrations/retrievers
https://python.langchain.com/docs/modules/data_connection/retrievers/web_research
